# 4 - Ligand Manipulations and Energetic evaluations

With a deeper understanding of both the ligand and core manipulations possible in Architector, we want to dig into more complex ligands for which the structures are much less intuitive or obvious!

In this tutorial we will look at the ligand manipulation routines in Architector, covering:

**(A)** Suggestions for identifying coordination sites in highly-coordinated ligands for Architector.

**(B)** Evaluating the isolated ligand energetics as bound to the metal.

**(C)** Utilizing wrappers for [CREST](https://xtb-docs.readthedocs.io/en/latest/crest.html) to sample gas-phase rotamers and compute the ligand pre-organization energies.

## For this tutorial we will study a commonly used chelating agent, Ethylenediaminetetraacetic acid (EDTA), bound to Ca<sup>2+</sup> 

Starting from loading in the ligand using our Architector utilites:

In [ ]:
from architector import (build_complex,
                        view_structures)

First - we get the EDTA smiles from [Wikipedia](https://en.wikipedia.org/wiki/Ethylenediaminetetraacetic_acid) and convert to ASE atoms.

In [ ]:
edta_smiles = 'OC(=O)CN(CCN(CC(O)=O)CC(O)=O)CC(O)=O'

In [ ]:
view_structures('CC')

Now we can visualize and identify the coordinating atoms.

## For (A), Here, note that for the ligand coordList with carboxylates, select oxygens without hydrogens bound, or deprotonated oxygens

In [ ]:
edta_smiles

In [ ]:
view_structures(edta_smiles,labelinds=True,w=500,h=500) 

In [ ]:
view_structures(edta_smiles,labelinds=True,w=500,h=500) 

Selecting these specific atoms gives a coordList for EDTA of:

In [ ]:
edta_coordList = [4,7,11,15,19,2]

And we now have enough to define an input dictionary for Architector!

In [ ]:
# Build the complex using some educated guesses for the geometries
inputDict = {
    'core':{'metal':'Ca',
            'coreType':'octahedral'}, # We have already down-selected for a likley correct core geometry
    'ligands':[
        {'smiles':edta_smiles,
        'coordList':edta_coordList,
        'ligType':'hexa_octahedral'} # Here, we have already assigned the most likely geometry manually
              ],
    'parameters':{'relax':True}
}
inputDict

Looks good! Ready for building

In [ ]:
out = build_complex(inputDict)

And a quick visual check reveals a fairly sensible structure:

In [ ]:
view_structures(out)

## Now for (B), we can isolate the ligand from the Ca2+ metal center!

This is easiestly done using the ASE atoms object from the output dictionary

In [ ]:
# Pull out the ase atoms and isolated ligand ase atoms from the lowest-energy conformer!
key = list(out.keys())[0]
full_atoms = out[key]['ase_atoms']

Now simply remove the Ca atom, this is easily done since Architector will always put the metal at the first index (0)!

In [ ]:
lig_atoms = full_atoms.copy()
del lig_atoms[0] # Remove the Ca
lig_atoms # Verify the Ca is gone

Note that there is no Ca in the symbols string of the atoms! We can also visualize the new ligand structure.

In [ ]:
view_structures(lig_atoms) # View ligand with metal removed.

Looks good! 

### Since we have removed the metal center, the charge and spin state of isolated ligand will be different and a new calculator will be needed. 

For this we import an additional utility for assigning a new ASE/XTB calculator:

In [ ]:
from architector.io_calc import CalcExecutor # Import architector calculator 

Now we can set the [GFN2-xTB](https://pubs.acs.org/doi/10.1021/acs.jctc.8b01176) method level of theory to the lig_atoms.
Making evalulation of the energy quite quick:

In [ ]:
lig_energy = CalcExecutor(lig_atoms,method='GFN2-xTB').energy
print('Ligand energy (eV):',lig_energy)

# For (C), we have developed a wrapper for the CREST utility to sample configurations of the EDTA without the metal!

Here we add just one more import:

In [ ]:
from architector.io_conformers import (crest_conformers,obmol_conformers)

Now, running crest is just one line. 

## NOTE THAT SOMETIMES this block fails due to CREST. Sometimes re-running the tutorial can fix the problem.

In [ ]:
# Slowest block in any tutorial! - likely minutes at a time depending on your CPU
# ligand_conformers, xtb_energies = crest_conformers(edta_smiles)

# Alternatively - Using the obmol conformer routine can be more stable and fast for isolated ligands! 

In [ ]:
## Slowest block in any tutorial! - likely minutes at a time depending on your CPU
ligand_conformers, xtb_energies = obmol_conformers(edta_smiles,obmol_total_confs=1000)

Once this finished (which can be quite slow!) we should have a list of EDTA conformers (ligand_conformers)
and their energies (xtb_energies). 

We can view what the lowest-energy conformers look like:

In [ ]:
# Note that very different conformers from the bound ligand are formed by sampling!
view_structures(ligand_conformers[0:10])
print('Minimum-Energy conformer!')
view_structures(ligand_conformers[xtb_energies.tolist().index(min(xtb_energies))])

And the lowest-energy crest conformer can be used to estimate the pre-organization energy!

In [ ]:
pre_org_energy = min(xtb_energies) - lig_energy
print('Pre-organization energy (eV):', pre_org_energy)

### This indicates the gas-phase lowest-energy EDTA conformer is ~2 eV
### lower in energy than the equivalent Ca-bound EDTA conformer.

# Conclusions

In this tutorial we looked at the ligand manipulation routines in Architector, covering:

**(A)** Suggestions for identifying coordination sites in highly-coordinated ligands for Architector.

**(B)** Evaluating the isolated ligand energetics as bound to the metal.

**(C)** Utilizing wrappers for [CREST](https://xtb-docs.readthedocs.io/en/latest/crest.html) to sample gas-phase rotamers and compute the ligand pre-organization energies.